In [ ]:
# 定义下载相关函数
import os

def aria2(url, filename, d):
    !aria2c --console-log-level=error -c -x 16 -s 16 {url} -o {filename} -d {d}

def download_from_oss(url, filename, save_dir):
    url_prefix = {
        "cn-shanghai": "http://pai-vision-data-sh.oss-cn-shanghai-internal.aliyuncs.com",
        "cn-hangzhou": "http://pai-vision-data-hz2.oss-cn-hangzhou-internal.aliyuncs.com",
        "cn-shenzhen": "http://pai-vision-data-sz.oss-cn-shenzhen-internal.aliyuncs.com",
        "cn-beijing": "http://pai-vision-data-bj.oss-cn-beijing-internal.aliyuncs.com",
        "cn-wulanchabu": "https://pai-vision-data-wlcb.oss-cn-wulanchabu-internal.aliyuncs.com",
        "ap-southeast-1": "http://pai-vision-data-ap-southeast.oss-ap-southeast-1-internal.aliyuncs.com" 
    }
    dsw_region = os.environ.get("dsw_region")

    prefix = url_prefix[dsw_region] if dsw_region in url_prefix else "http://pai-vision-data-sh.oss-cn-shanghai.aliyuncs.com"
    url = os.path.join(prefix,url,filename)
    print(url)
    print(dsw_region)


    print(f'Download from {url}')
    !aria2c --console-log-level=error -c -x 16 -s 16 {url} -o {filename} -d {save_dir}

    if filename.endswith('.tar.gz') or filename.endswith('.tar'):
        saved_filepath = os.path.join(save_dir, filename)
        print('Start Unzipping...')
        !tar -xf $saved_filepath -C $save_dir
        print('Done')

        try:
            os.remove(saved_filepath)
            print(f'Removed original file: {saved_filepath}')
        except OSError as e:
            print(f"Error: {saved_filepath} : {e.strerror}")


def check_files_exists_and_download(data_path, download_filenames):    
    filenames = [
        os.path.join(data_path, f"models/Diffusion_Transformer/{name}") for name in download_filenames
    ]

    for download_filename, filename in zip(download_filenames, filenames): 
        if os.path.exists(filename) or os.path.exists(filename.replace('.tar','').replace('.gz','')):
            print('Exists. ', filename)
            continue
        save_dir = os.path.dirname(filename)
        os.makedirs(save_dir, exist_ok=True)
        print(f"Start Downloading: {download_filename} to {save_dir}")
        download_from_oss('aigc-data/cogvideox_fun/models', download_filename, save_dir)

In [1]:
!git clone https://github.com/aigc-apps/EasyAnimate.git

Cloning into 'EasyAnimate'...
remote: Enumerating objects: 1603, done.
remote: Counting objects: 100% (660/660), done.
remote: Compressing objects: 100% (283/283), done.
remote: Total 1603 (delta 541), reused 377 (delta 377), pack-reused 943 (from 3)
Receiving objects: 100% (1603/1603), 4.30 MiB | 12.09 MiB/s, done.
Resolving deltas: 100% (1028/1028), done.


In [2]:
%cd /home/ubuntu/EasyAnimate
!pip install -r requirements.txt

/home/ubuntu/EasyAnimate
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 KB 953.3 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 KB 6.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 32.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 KB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 60.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 KB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 67.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 29.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
%cd /home/ubuntu/EasyAnimate
!mkdir -p datasets/internal_datasets/train 

/home/ubuntu/EasyAnimate


import os
data_path = os.getcwd()
download_from_oss('aigc-data/cogvideox_fun/code', 'code.tar.gz', data_path)

In [ ]:
# 下载模型到指定目录 约3min
import os
data_path = os.path.join(os.getcwd(),'CogVideoX-Fun')

# 下载预训练的模型
check_files_exists_and_download(data_path, ['CogVideoX-Fun-2b-InP.tar.gz'])

In [5]:
# 安装 huggingface_hub 库
!pip install huggingface_hub

from huggingface_hub import snapshot_download

# 指定模型仓库的 repo_id
repo_id = "alibaba-pai/EasyAnimateV5.1-12b-zh-InP"

# 指定下载到的目标目录
target_dir = "/home/ubuntu/EasyAnimate"

# 下载整个模型仓库的快照，指定缓存目录
model_dir = snapshot_download(repo_id=repo_id, cache_dir=target_dir)

print("模型已下载至：", model_dir)

Defaulting to user installation because normal site-packages is not writeable


/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 30 files: 100%|██████████| 30/30 [07:45<00:00, 15.52s/it]

模型已下载至： /home/ubuntu/EasyAnimate/models--alibaba-pai--EasyAnimateV5.1-12b-zh-InP/snapshots/c0787d7ac5d75565255072e5889362d9c0b05509


In [ ]:
#记住，把下载好的文件夹models--alibaba-pai--CogVideoX-Fun-V1.1-5b-InP直接重命名为 models/models/Diffusion_Transformer/CogVideoX-Fun-2b-InP，复制太慢了，要一小时。
!cp -r /home/ubuntu/CogVideoX-Fun/models/models--alibaba-pai--CogVideoX-Fun-V1.1-5b-InP/snapshots/b3798d82878e57443314b29d73633a165dd4c008/ /home/ubuntu/CogVideoX-Fun/models/Diffusion_Transformer/

In [ ]:
!ls -R /home/ubuntu/CogVideoX-Fun/models/Diffusion_Transformer/CogVideoX-Fun-2b-InP


In [10]:
!cd /home/ubuntu/EasyAnimate && python app.py

2025-03-22 12:39:31.464849: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-22 12:39:32.579925: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742647173.005830    2647 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742647173.135181    2647 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-22 12:39:34.152103: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [6]:
folder_path = '/home/ubuntu/EasyAnimate/models'  # 替换为你要删除的文件夹路径
!rm -r "$folder_path"
print(f"文件夹 '{folder_path}' 及其内容已成功删除。")

文件夹 '/home/ubuntu/EasyAnimate/models' 及其内容已成功删除。


In [7]:
!cd /home/ubuntu/EasyAnimate
#第一步根据下载模型的不同，需要自己重命名
!mv models--alibaba-pai--EasyAnimateV5.1-12b-zh-InP models 
!cd /home/ubuntu/EasyAnimate/models
!mv snapshots Diffusion_Transformer
!cd /home/ubuntu/EasyAnimate/models/Diffusion_Transformer
!mv snapshots EasyAnimateV5.1-12b-zh-InP
#最后一步根据下载模型的不同，需要自己重命名

mv: cannot stat 'snapshots': No such file or directory
/bin/bash: line 1: cd: /home/ubuntu/EasyAnimate/models/Diffusion_Transformer: No such file or directory
mv: cannot stat 'snapshots': No such file or directory


In [ ]:
folder_path = '/home/ubuntu/EasyAnimate/models/'  # 替换为你要删除的文件夹路径
!rm -r "$folder_path"
print(f"文件夹 '{folder_path}' 及其内容已成功删除。")

In [ ]:
!cp -r /root/.cache/huggingface/hub/models--alibaba-pai--CogVideoX-Fun-V1.5-5b-InP /home/ubuntu/EasyAnimate/models/Diffusion_Transformer/CogVideoX-Fun-V1.1-2b-InP

In [ ]:
!cd /home/ubuntu/EasyAnimate && python /home/ubuntu/EasyAnimate/predict_i2v.py

In [ ]:
# 下载示例数据
data_path = os.path.join(os.getcwd(),'EasyAnimate/datasets')
download_from_oss('aigc-data/easyanimate/data', 'data.tar.gz', data_path)

In [ ]:
# train lora model
# 您可修改 scripts/train_lora.sh 中的相关参数（如增大训练epoch数 num_train_epochs）来达到更好的性能。
#注意metadata.json是image还是video
!cd /home/ubuntu/EasyAnimate && sh scripts/train_lora.sh

In [ ]:
#修改train.sh保持所有的参数和文件名一致

In [14]:
# train model
# 您可修改 scripts/train.sh 中的相关参数（如增大训练epoch数 num_train_epochs）来达到更好的性能。
# 删去trainmodel=inpaint
#注意metadata.json是image还是video
#这是训练DiT的，DiT是什么参考：https://docs.google.com/document/d/1Ah6N8lZg4uIOAf0b4f8AoIeCjYcF1fJFYqTi7upDnKE/edit?tab=t.0
!cd /home/ubuntu/EasyAnimate && sh scripts/train.sh

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-03-22 13:10:20.760919: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-22 13:10:20.782693: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742649020.808248    3487 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been

In [12]:
!pip install --upgrade jinja2


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 KB 1.8 MB/s eta 0:00:00a 0:00:01


In [ ]:
!pip install xformers

In [1]:
!nvidia-smi

Failed to initialize NVML: Driver/library version mismatch


In [3]:
!pip install cuda

ERROR: Could not find a version that satisfies the requirement cuda (from versions: none)
ERROR: No matching distribution found for cuda


In [4]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Mon_Apr__3_17:16:06_PDT_2023
Cuda compilation tools, release 12.1, V12.1.105
Build cuda_12.1.r12.1/compiler.32688072_0


In [ ]:
# 移动训练好的LoRA模型到 EasyAnimate/models/Personalized_Model 目录中
!cd EasyAnimate && mkdir -p models/Personalized_Model/ && mv output_dir/checkpoint-100.safetensors models/Personalized_Model/

In [ ]:
# 启动WebUI 在 Select LoRA model (选择LoRA模型[非必需]) 处选择训练好的LoRA模型
!cd EasyAnimate && python app.py

In [ ]:
!unzip /home/ubuntu/EasyAnimate/datasets/00033-3725054118.zip -d /home/ubuntu/EasyAnimate/datasets/internal_datasets/train

In [ ]:
!unzip /home/ubuntu/EasyAnimate/datasets/train-20250321T172108Z-001.zip -d /home/ubuntu/EasyAnimate/datasets/internal_datasets

In [ ]:
datasets/panda_70m/videos/data/

In [ ]:
%cd /home/ubuntu/EasyAnimate/easyanimate/video_caption/
!sh ./scripts/stage_1_video_splitting.sh

In [ ]:
%cd /home/ubuntu/EasyAnimate/easyanimate/video_caption
!pip install -r requirements.txt

In [ ]:
%pwd

In [ ]:
%cat ./scripts/stage_1_video_splitting.sh

In [34]:
%cd /home/ubuntu/EasyAnimate/easyanimate/video_caption/
!sh ./scripts/stage_1_video_splitting.sh

/workspace/EasyAnimate/easyanimate/video_caption


/venv/main/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


2025-03-22 00:58:23,156 - VideoCaption - INFO - 4 files in total. Save the result to /workspace/EasyAnimate/easyanimate/video_caption/datasets/panda_70m/videos/meta_file_info.jsonl.
2025-03-22 00:58:23,837 - VideoCaption - INFO - -1hTK0wGrTg.mp4 has been processed.
2025-03-22 00:58:23,837 - VideoCaption - INFO - -BgGhZs4K4w.mp4 has been processed.
2025-03-22 00:58:23,837 - VideoCaption - INFO - -1urvnqfnfk.mp4 has been processed.
2025-03-22 00:58:23,837 - VideoCaption - INFO - -AM0oWK3RBA.mp4 has been processed.
100%|██████████████████████████████████████████| 4/4 [00:00<00:00, 13325.83it/s]
2025-03-22 00:58:24,672 - VideoCaption - INFO - Save the gathered single jsonl file to /workspace/EasyAnimate/easyanimate/video_caption/datasets/panda_70m/videos/meta_scene_info.jsonl.
2025-03-22 00:58:25,155 - VideoCaption - INFO - Filter 0 videos with resolution smaller than 262144.0.
100%|█████████████████████████████████████████████| 4/4 [01:38<00:00, 24.55s/it]
